In [2]:
import os
import pathlib
import tarfile

import boto3
from botocore.handlers import disable_signing

from scipy.io import loadmat
from scipy.misc import imread

import numpy as np
from skimage.transform import resize


def list_files(base_path, validExts=(".jpg", ".jpeg", ".png", ".bmp"), contains=None):
    # loop over the directory structuref
    for (rootDir, dirNames, filenames) in os.walk(base_path):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename).replace(" ", "\\ ")
                yield imagePath


class BSDS500(object):
    BUCKET = 'i008data'
    FN = 'BSR_bsds500.tgz'
    STORE_FN = os.path.join(os.environ['HOME'], 'BSR.tgz')
    HOME = os.environ['HOME']

    def __init__(self, path_to_bsds=None, images_to_gray=False, target_size=None, masks_to_binary=True):
        print (self.STORE_FN)
        if not path_to_bsds:
            self.BSDS_BASE = self.get_bsds()
        else:
            self.BSDS_BASE = path_to_bsds
            
        print(self.BSDS_BASE)

        self.images_to_gray = images_to_gray
        self.target_size = target_size
        self.masks_to_binary = masks_to_binary

        self.TRAIN_PATH = os.path.join(self.BSDS_BASE, 'BSDS500/data/images/train/')
        print (self.TRAIN_PATH)
        self.TEST_PATH = os.path.join(self.BSDS_BASE, 'BSDS500/data/images/test/')
        self.VALID_PATH = os.path.join(self.BSDS_BASE, 'BSDS500/data/images/val/')
        self.GROUND_TRUTH_TRAIN = os.path.join(self.BSDS_BASE, 'BSDS500/data/groundTruth/train/')
        self.GROUND_TRUTH_TEST = os.path.join(self.BSDS_BASE, 'BSDS500/data/groundTruth/test/')
        self.GROUND_TRUTH_VALID = os.path.join(self.BSDS_BASE, 'BSDS500/data/groundTruth/val/')

    def get_bsds(self):
        if not pathlib.Path(self.STORE_FN).exists():
            print("DOWNLOADING BSDS500 DATA BE PATIENT")
            s3_resource = boto3.resource('s3')
            s3_resource.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
            bucket = s3_resource.Bucket(self.BUCKET)
            bucket.download_file(self.FN, self.STORE_FN)
            
        ds_dir = self.STORE_FN.split('.')[0]
        
        if not pathlib.Path(ds_dir).is_dir():
            tar = tarfile.open(self.STORE_FN)
            tar.extractall(self.HOME)

#         dir_path = os.path.dirname(os.path.realpath(__file__))
#         return os.path.join(dir_path, self.STORE_FN.split('.')[0])
        return (self.STORE_FN.split('.')[0])

    def load_ground_truth(self, gt_path):
        ground_truth_paths = sorted(list(list_files(gt_path, validExts=('.mat'))))
        file_id = []
        cnts = []
        sgmnts = []

        for gt_path in ground_truth_paths:
            file_name = os.path.basename(gt_path).split('.')[0]
            gt = loadmat(gt_path)
            gt = gt['groundTruth'][0]
            for annotator in gt:
                contours = annotator[0][0][1]  # 1-> contours
                segments = annotator[0][0][0]  # 0 -> segments
                if self.target_size:
                    contours = resize(contours.astype(float), output_shape=self.target_size)
                    segments = resize(segments, output_shape=self.target_size)

                if self.masks_to_binary:
                    contours[contours > 0] = 1

                file_id.append(file_name)
                cnts.append(contours)
                sgmnts.append(segments)

        cnts = np.concatenate([np.expand_dims(a, 0) for a in cnts])
        sgmnts = np.concatenate([np.expand_dims(a, 0) for a in sgmnts])
        cnts = cnts[..., np.newaxis]
        sgmnts = sgmnts[..., np.newaxis]

        return file_id, cnts, sgmnts

    def load_images(self, list_of_files):
        processed_images = []
        for i, f in enumerate(list_of_files):

            if self.images_to_gray:
                im = imread(f, mode='L')
            else:
                im = imread(f)

            if self.target_size:
                im = resize(im, output_shape=self.target_size)

            processed_images.append(np.expand_dims(im, 0))

        processed_images = np.concatenate(processed_images)

        if self.images_to_gray:
            processed_images = processed_images[..., np.newaxis]

        return processed_images

    def get_train(self):
        file_ids, cnts, sgmnts = self.load_ground_truth(self.GROUND_TRUTH_TRAIN)
        image_paths = [self.TRAIN_PATH + f_id + '.jpg' for f_id in file_ids]
        images = self.load_images(image_paths)

        return file_ids, cnts, sgmnts, images

    def get_test(self):
        file_ids, cnts, sgmnts = self.load_ground_truth(self.GROUND_TRUTH_TEST)
        image_paths = [self.TEST_PATH + f_id + '.jpg' for f_id in file_ids]
        images = self.load_images(image_paths)

        return file_ids, cnts, sgmnts, images

    def get_val(self):
        file_ids, cnts, sgmnts = self.load_ground_truth(self.GROUND_TRUTH_VALID)
        image_paths = [self.VALID_PATH + f_id + '.jpg' for f_id in file_ids]
        images = self.load_images(image_paths)
        
        return file_ids, cnts, sgmnts, images

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
 
K.set_image_dim_ordering('tf')
 
def get_unet(img_rows, img_cols, channels):
    inputs = Input((img_rows, img_cols, channels))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
 
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
 
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
 
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
 
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
 
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
 
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
 
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
 
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
 
    model = Model(inputs=[inputs], outputs=[conv10])
 
    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy')
    return model

In [ ]:
TARGET_SHAPE = (256, 256) 
bsds = BSDS500(target_size=TARGET_SHAPE)

In [ ]:
ids, contours_train, sgmnts, train_images = bsds.get_train()
ids, contours_test, sgmnts, test_images = bsds.get_test()
 
C = np.concatenate([contours_test, contours_train])
I = np.concatenate([test_images, train_images])

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
# csv_callback = CSVLogger('history.log', append=True)
unet = get_unet(256, 256, 3)
history = unet.fit(I, C,verbose=1, epochs=10, validation_split=0.1) #, callbacks=[csv_callback])

In [ ]:
unet.save('test_unet_model_10.h5')


In [ ]:
unet = get_unet(256, 256, 3)
unet.load_weights('test_unet_model_10.h5')

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
ids, cnts, sgmtns, images = bsds.get_val()

In [ ]:
preds = unet.predict(images[:])
preds = preds[:, :, :, 0]
cnts_1d = cnts[:,:,0]
images_uint8 = (images*255).round().astype('uint8')

In [ ]:
import matplotlib.pyplot as plt
f, axarr = plt.subplots(3, 4, figsize=(15,15))
for i, (original, pred, blurr) in enumerate(list(zip(images, preds, images_uint8))[410:450:10]):
#     blurr = blurr_canny(blurr)
    
    orig_row = axarr[0, i]
    orig_row.imshow(original)
    orig_row.axis('off')
  
    pred_row = axarr[1, i]
    pred_row.imshow(pred)
    pred_row.axis('off')
    
    canny_row = axarr[2, i]
    canny_row.imshow(blurr)
    canny_row.axis('off')
    
    orig_row.set_title('original')
    pred_row.set_title('deep edge prediction')
    canny_row.set_title('canny edge detection')

    plt.subplots_adjust(hspace=0, wspace=0.05)

In [ ]:
def predict_custom_image(image=None, model=None):
    if isinstance(image, str):
        im = imread(image)
    else:
        im = image
    im = im[:,:,:3]    
    print(im.shape)
    im = (im-np.min(im))/(np.max(im)-np.min(im))
    if len(im.shape) == 2:
        im = to_rgb1(im)

    target_size = model.input.__dict__['_keras_shape'][1:-1]

    im_resize = resize(im, target_size)
    im = np.expand_dims(im_resize, 0)
    
    preds = model.predict(im)
    pred = preds[:, :, :, 0][0]
    return pred

In [ ]:
output_file = 'test1_orig.png'
plt.figure(figsize=(10,10))
p = predict_custom_image(image=output_file, model=unet)

coins = imread(output_file)
coin = coins[:,:,:3]
print (coin.shape)
print(coins.shape)
plt.subplot(121)
plt.axis('off')
plt.imshow(resize(coins, (256,256)), cmap='gray')
plt.title('coins orig')
plt.subplot(122)
plt.axis('off')
plt.title('deep edge detection')
plt.imshow(p)
plt.imsave(output_file[0:output_file.find('/')+1] + 'pred_' + output_file[output_file.find('/')+1:], p)

In [ ]:
plt.imsave('test2.png', p)